In [1]:
#Created by Meron Sheffer and Emanuele Dalla Torre, Bar-Ilan University, all rights reserved, December 2020
#initialization

import numpy as np
from math import *
from qiskit import *
from qiskit.circuit import *
from qiskit.compiler import transpile, assemble
from qiskit.result import *
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.tools.monitor import *
from qiskit.transpiler import *
from qiskit.ignis.mitigation.measurement import (complete_meas_cal,CompleteMeasFitter)
from qiskit.quantum_info import *
from qiskit.providers.aer import QasmSimulator


IBMQ.load_account()
provider= IBMQ.get_provider('ibm-q')

#**********PARAMETERS TO BE CHANGED****************
qcomp= provider.get_backend('ibmq_16_melbourne')
pq=[4,5,6,8,9,10] #physical qubits 

In [2]:
#CREATE ERROR MITIGATION MATRIX FOR EACH PLAYER INDIVIDUALLY
#created by the dark arts of linear algebra
n=2
qr=QuantumRegister(n)
cr=ClassicalRegister(n)
qc=QuantumCircuit(qr,cr)
meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')
job1= execute(meas_calibs,backend=qcomp,optimization_level=0,initial_layout={qr[0]:pq[0],qr[1]:pq[1]})
job_monitor(job1)
cal_result=job1.result()
meas_fitter = CompleteMeasFitter(cal_result, state_labels, circlabel='mcal')
m1=meas_fitter.cal_matrix
job1= execute(meas_calibs,backend=qcomp,optimization_level=0,initial_layout={qr[0]:pq[2],qr[1]:pq[3]})#,qr[4]:9,qr[5]:10})
job_monitor(job1)
cal_result=job1.result()
meas_fitter = CompleteMeasFitter(cal_result, state_labels, circlabel='mcal')
m2=meas_fitter.cal_matrix
job1= execute(meas_calibs,backend=qcomp,optimization_level=0,initial_layout={qr[0]:pq[4],qr[1]:pq[5]})
job_monitor(job1)
cal_result=job1.result()
meas_fitter = CompleteMeasFitter(cal_result, state_labels, circlabel='mcal')
m3=meas_fitter.cal_matrix              
matrix=np.zeros((64,64))
for i in range(64):
    for j in range(64):
            matrix[i,j]=m1[i%4,j%4]*m2[int(i/4)%4,int(j/4)%4]*m3[int(i/16)%4,int(j/16)%4]
Cmatrix=np.linalg.inv(matrix)
def calc_real_y(t,ys):
    x=np.dot(Cmatrix[t],ys)
    return x

Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run


In [3]:
#NORMALIZATION
def rewrite_y(inp):
    ys=[0]*64
    i=0
    for x,y in inp.items():
        ys[i]=y
        i+=1
    i=0
    count=0
    newinp={}
    for x,y in inp.items():
        newinp[x]=calc_real_y(i,ys)
        if (newinp[x]>0):
            count+=newinp[x]
        else:
            newinp[x]=0
        i+=1
    newinp.update((x, y*1024/count) for x, y in newinp.items())
    return newinp


In [4]:
#Prepare for the full calculation
n=6
qr=QuantumRegister(n)
cr=ClassicalRegister(n)
il = {qr[i]:pq[i] for i in range(6)} #intial_layout

In [5]:
#for 000
qc=QuantumCircuit(qr,cr)
for i in range (int(n/2)):
    qc.h(2*i)
    qc.cx (2*i,(2*i-1)%6)
    qc.cx (2*i,(2*i+1)%6)
    qc.h(2*i)
for i in range (n):
    qc.measure(i,i)

job1= execute(qc,backend=qcomp,optimization_level=0 ,initial_layout=il)
job_monitor(job1)
resultexp=job1.result()
inp=resultexp.get_counts()
win1no=0
for x,y in inp.items():
    a=0
    b=0
    c=0
    for i in range(3):
        a+=int(x[2*i])
        b+=int(x[2*i+1])
        c+=int(x[2*i])^int(x[2*i+1])
    if (a%2==0 and b%2==0 and c%2==0):
        win1no+=y
inp=rewrite_y(resultexp.get_counts())
win1=0
for x,y in inp.items():
    a=0
    b=0
    c=0
    for i in range(3):
        a+=int(x[2*i])
        b+=int(x[2*i+1])
        c+=int(x[2*i])^int(x[2*i+1])
    if (a%2==0 and b%2==0 and c%2==0):
        win1+=y
print ('win percent:',win1no*100/1024,'% --> ',win1*100/1024,'%')

Job Status: job has successfully run
win percent: 62.59765625 % -->  72.18518619505112 %


In [6]:
#for 001
qc=QuantumCircuit(qr,cr)

for i in range (int(n/2)):
    qc.h(2*i)
    qc.cx (2*i,(2*i-1)%6)
    qc.cx (2*i,(2*i+1)%6)
qc.rz(pi/2,4)
for i in range (n):
    if i%2==0:
        qc.h(i)
    qc.measure(i,i)

job1= execute(qc,backend=qcomp,optimization_level=0,initial_layout=il)
job_monitor(job1)
resultexp=job1.result()
inp=resultexp.get_counts()
win2no=0
for x,y in inp.items():
    a1=int(x[5])
    b1=int(x[4])
    c1=(int(x[4])+int(x[5]))%2
    a2=int(x[3])
    b2=int(x[2])
    c2=(int(x[2])+int(x[3]))%2
    d3=(int(x[0])+int(x[1]))%2
    b3=int(x[0])
    e3=int(x[1])
    if ((b1+b2+b3)%2==0 and (a1+c1+a2+c2+d3+e3)%2==0):
        win2no+=y
inp=rewrite_y(resultexp.get_counts())
win2=0
for x,y in inp.items():
    a1=int(x[5])
    b1=int(x[4])
    c1=(int(x[4])+int(x[5]))%2
    a2=int(x[3])
    b2=int(x[2])
    c2=(int(x[2])+int(x[3]))%2
    d3=(int(x[0])+int(x[1]))%2
    b3=int(x[0])
    e3=int(x[1])
    if ((b1+b2+b3)%2==0 and (a1+c1+a2+c2+d3+e3)%2==0):
        win2+=y
print ('win percent:',win2no*100/1024,'% --> ',win2/10.24,'%')


Job Status: job has successfully run
win percent: 78.22265625 % -->  86.3737213268314 %


In [7]:
#for 011

qc=QuantumCircuit(qr,cr)
for i in range (int(n/2)):
    qc.h(2*i)
    qc.cx (2*i,(2*i-1)%6)
    qc.cx (2*i,(2*i+1)%6)
qc.rz(pi/2,2)
qc.rz(pi/2,4)
for i in range (n):
    if i%2==0:
        qc.h(i)
    qc.measure(i,i)

job1= execute(qc,backend=qcomp,optimization_level=0,initial_layout=il)
job_monitor(job1)
resultexp=job1.result()
inp=resultexp.get_counts()
win3no=0
for x,y in inp.items():
    a1=int(x[5])
    b1=int(x[4])
    c1=(int(x[4])+int(x[5]))%2
    d2=(int(x[3])+int(x[2]))%2
    b2=int(x[2])
    e2=int(x[3])
    d3=(int(x[1])+int(x[0]))%2
    b3=int(x[0])
    e3=int(x[1])
    if ((b1+b2+b3)%2==0 and (a1+c1+d2+e2+d3+e3)%2==0 and (d2+e3+a1)%2==1):
        win3no+=y
win3=0
inp=rewrite_y(resultexp.get_counts())
for x,y in inp.items():
    a1=int(x[5])
    b1=int(x[4])
    c1=(int(x[4])+int(x[5]))%2
    d2=(int(x[3])+int(x[2]))%2
    b2=int(x[2])
    e2=int(x[3])
    d3=(int(x[1])+int(x[0]))%2
    b3=int(x[0])
    e3=int(x[1])
    if ((b1+b2+b3)%2==0 and (a1+c1+d2+e2+d3+e3)%2==0 and (d2+e3+a1)%2==1):
        win3+=y
print ('win percent:',win3no*100/1024,'% --> ',win2/10.24,'%')


Job Status: job has successfully run
win percent: 61.62109375 % -->  86.3737213268314 %


In [8]:
#for 111
qc=QuantumCircuit(qr,cr)
for i in range (int(n/2)):
    qc.h(2*i)
    qc.cx (2*i,(2*i-1)%6)
    qc.cx (2*i,(2*i+1)%6)
qc.rz(pi/2,0)
qc.rz(pi/2,2)
qc.rz(pi/2,4)
for i in range (n):
    if i%2==0:
        qc.h(i)
    qc.measure(i,i)
qc.draw()

job1= execute(qc,backend=qcomp,optimization_level=0 ,initial_layout=il)
job_monitor(job1)
resultexp=job1.result()
inp=resultexp.get_counts()
win4no=0
for x,y in inp.items():
    d1=(int(x[4])+int(x[5]))%2
    b1=int(x[4])
    e1=int(x[5])
    d2=(int(x[3])+int(x[2]))%2
    b2=int(x[2])
    e2=int(x[3])
    d3=(int(x[1])+int(x[0]))%2
    b3=int(x[0])
    e3=int(x[1])
    if ((b1+b2+b3)%2==0 and (d1+e1+d2+e2+d3+e3)%2==0):
        win4no+=y
inp=rewrite_y(resultexp.get_counts())
win4=0
for x,y in inp.items():
    d1=(int(x[4])+int(x[5]))%2
    b1=int(x[4])
    e1=int(x[5])
    d2=(int(x[3])+int(x[2]))%2
    b2=int(x[2])
    e2=int(x[3])
    d3=(int(x[1])+int(x[0]))%2
    b3=int(x[0])
    e3=int(x[1])
    if ((b1+b2+b3)%2==0 and (d1+e1+d2+e2+d3+e3)%2==0):
        win4+=y
print ('win percent:',win4no*100/1024,'% --> ',win4/10.24,'%')


Job Status: job has successfully run
win percent: 76.7578125 % -->  85.26673573418645 %


In [9]:
#for 010

qc=QuantumCircuit(qr,cr)
for i in range (int(n/2)):
    qc.h(2*i)
    qc.cx (2*i,(2*i-1)%6)
    qc.cx (2*i,(2*i+1)%6)
qc.rz(pi/2,2)
for i in range (n):
    if i%2==0:
        qc.h(i)
    qc.measure(i,i)

job1= execute(qc,backend=qcomp,optimization_level=0,initial_layout=il)
job_monitor(job1)
resultexp=job1.result()
inp=resultexp.get_counts()
win5no=0
for x,y in inp.items():
    a1=int(x[5])
    b1=int(x[4])
    c1=(int(x[4])+int(x[5]))%2
    a3=int(x[1])
    b3=int(x[0])
    c3=(int(x[1])+int(x[0]))%2
    d2=(int(x[3])+int(x[2]))%2
    b2=int(x[2])
    e2=int(x[3])
    if ((b1+b2+b3)%2==0 and (a1+c1+a3+c3+d2+e2)%2==0):
        win5no+=y
inp=rewrite_y(resultexp.get_counts())
win5=0
for x,y in inp.items():
    a1=int(x[5])
    b1=int(x[4])
    c1=(int(x[4])+int(x[5]))%2
    a3=int(x[1])
    b3=int(x[0])
    c3=(int(x[1])+int(x[0]))%2
    d2=(int(x[3])+int(x[2]))%2
    b2=int(x[2])
    e2=int(x[3])
    if ((b1+b2+b3)%2==0 and (a1+c1+a3+c3+d2+e2)%2==0):
        win5+=y
print ('win percent:',win5no*100/1024,'% --> ',win5/10.24,'%')


Job Status: job has successfully run
win percent: 78.61328125 % -->  85.79413458398012 %


In [10]:
#for 100

qc=QuantumCircuit(qr,cr)
for i in range (int(n/2)):
    qc.h(2*i)
    qc.cx (2*i,(2*i-1)%6)
    qc.cx (2*i,(2*i+1)%6)
qc.rz(pi/2,0)
for i in range (n):
    if i%2==0:
        qc.h(i)
    qc.measure(i,i)
    
job1= execute(qc,backend=qcomp,optimization_level=0 ,initial_layout=il)
job_monitor(job1)
resultexp=job1.result()
inp=rewrite_y(resultexp.get_counts())
win6no=0
for x,y in inp.items():
    a3=int(x[1])
    b3=int(x[0])
    c3=(int(x[1])+int(x[0]))%2
    a2=int(x[3])
    b2=int(x[2])
    c2=(int(x[3])+int(x[2]))%2
    d1=(int(x[4])+int(x[5]))%2
    b1=int(x[4])
    e1=int(x[5])
    if ((b1+b2+b3)%2==0 and (a3+c3+a2+c2+d1+e1)%2==0):
        win6no+=y
inp=rewrite_y(resultexp.get_counts())
win6=0
for x,y in inp.items():
    a3=int(x[1])
    b3=int(x[0])
    c3=(int(x[1])+int(x[0]))%2
    a2=int(x[3])
    b2=int(x[2])
    c2=(int(x[3])+int(x[2]))%2
    d1=(int(x[4])+int(x[5]))%2
    b1=int(x[4])
    e1=int(x[5])
    if ((b1+b2+b3)%2==0 and (a3+c3+a2+c2+d1+e1)%2==0):
        win6+=y
print ('win percent:',win6no*100/1024,'% --> ',win6/10.24,'%')


Job Status: job has successfully run
win percent: 86.87194650361192 % -->  86.87194650361192 %


In [11]:
#for 101
qc=QuantumCircuit(qr,cr)
for i in range (int(n/2)):
    qc.h(2*i)
    qc.cx (2*i,(2*i-1)%6)
    qc.cx (2*i,(2*i+1)%6)
qc.rz(pi/2,0)
qc.rz(pi/2,4)
for i in range (n):
    if i%2==0:
        qc.h(i)
    qc.measure(i,i)

job1= execute(qc,backend=qcomp,optimization_level=0,initial_layout=il)
job_monitor(job1)
resultexp=job1.result()

inp=resultexp.get_counts()
win7no=0
for x,y in inp.items():
    a2=int(x[3])
    b2=int(x[2])
    c2=(int(x[2])+int(x[3]))%2
    d1=(int(x[4])+int(x[5]))%2
    b1=int(x[4])
    e1=int(x[5])
    d3=(int(x[0])+int(x[1]))%2
    b3=int(x[0])
    e3=int(x[1])
    if (((b1+b2+b3)%2==0) and ((a2+c2+d1+e1+d3+e3)%2==0) and ((d3+e1+a2)%2==1)):
        win7no+=y
inp=rewrite_y(resultexp.get_counts())
win7=0
for x,y in inp.items():
    a2=int(x[3])
    b2=int(x[2])
    c2=(int(x[2])+int(x[3]))%2
    d1=(int(x[4])+int(x[5]))%2
    b1=int(x[4])
    e1=int(x[5])
    d3=(int(x[0])+int(x[1]))%2
    b3=int(x[0])
    e3=int(x[1])
    if (((b1+b2+b3)%2==0) and ((a2+c2+d1+e1+d3+e3)%2==0) and ((d3+e1+a2)%2==1)):
        win7+=y
print ('win percent:',win7no*100/1024,'% --> ',win7/10.24,'%')


Job Status: job has successfully run
win percent: 62.109375 % -->  72.5913188856319 %


In [12]:
#for 110
qc=QuantumCircuit(qr,cr)
for i in range (int(n/2)):
    qc.h(2*i)
    qc.cx (2*i,(2*i-1)%6)
    qc.cx (2*i,(2*i+1)%6)
qc.rz(pi/2,0)
qc.rz(pi/2,2)
for i in range (n):
    if i%2==0:
        qc.h(i)
    qc.measure(i,i)
win8=0

job1= execute(qc,backend=qcomp,optimization_level=0,initial_layout=il)
job_monitor(job1)
resultexp=job1.result()
inp=rewrite_y(resultexp.get_counts())
win8no=0
for x,y in inp.items():
    a3=int(x[1])
    b3=int(x[0])
    c3=(int(x[0])+int(x[1]))%2
    d2=(int(x[2])+int(x[3]))%2
    b2=int(x[2])
    e2=int(x[3])
    d1=(int(x[4])+int(x[5]))%2
    b1=int(x[4])
    e1=int(x[5])
    if ((b1+b2+b3)%2==0 and (a3+c3+d2+e2+d1+e1)%2==0 and (e2+d1+a3)%2==1):
        win8no+=y
inp=rewrite_y(resultexp.get_counts())
win8=0
for x,y in inp.items():
    a3=int(x[1])
    b3=int(x[0])
    c3=(int(x[0])+int(x[1]))%2
    d2=(int(x[2])+int(x[3]))%2
    b2=int(x[2])
    e2=int(x[3])
    d1=(int(x[4])+int(x[5]))%2
    b1=int(x[4])
    e1=int(x[5])
    if ((b1+b2+b3)%2==0 and (a3+c3+d2+e2+d1+e1)%2==0 and (e2+d1+a3)%2==1):
        win8+=y
print ('win percent:',win8no*100/1024,'% --> ',win8/10.24,'%')


Job Status: job has successfully run
win percent: 74.02619813256945 % -->  74.02619813256945 %


In [14]:
print("no:",win1no/10.24,win2no/10.24,win3no/10.24,win4no/10.24,win5no/10.24,win6no/10.24,win7no/10.24,win8no/10.24);
print("average no:",(win1no+win2no+win3no+win4no+win5no+win6no+win7no+win8no)/10.24/8,"%")
print("yes:",win1/10.24,win2/10.24,win3/10.24,win4/10.24,win5/10.24,win6/10.24,win7/10.24,win8/10.24);
print("average yes:",(win1+win2+win3+win4+win5+win6+win7+win8)/10.24/8,"%")

no: 62.59765625 78.22265625 61.62109375 76.7578125 78.61328125 86.87194650361192 62.109375 74.02619813256945
average no: 72.60250245452268 %
yes: 72.18518619505112 86.3737213268314 74.44884199339806 85.26673573418645 85.79413458398012 86.87194650361192 72.5913188856319 74.02619813256945
average yes: 79.69476041940756 %
